In [1]:
from PIL import Image
import numpy as np
import os

layerList = ["가", "나", "다", "라", "마", "바", "아", "자", "차", "카", "타", "파", "하"]

all_x_array = []
all_y_array = []

for dir in layerList:
    fileList = os.listdir(f"../dataset_dir/new_image/{dir}")

    for file in fileList:
        y_array = dict.fromkeys(layerList, 0) # 12000개의 str 값을 0으로 초기화하는 dict 생성

        imageData = Image.open(f"""../dataset_dir/new_image/{dir}/{file}""").convert("L") # 이미지 tensor 데이터 출력
        imageData = imageData.resize((50, 50)) # 이미지 업스케일링

        imageData = np.array(imageData)
        all_x_array.append(imageData.tolist())

        y_array[dir] = 1 # 사용자가 지정한 str 값을 1로 변경
        y_array = list(y_array.values())
        all_y_array.append(y_array)


In [2]:
x_array = np.array(all_x_array)
y_array = np.array(all_y_array)

In [3]:
x_array.shape

(80988, 50, 50)

In [4]:
y_array.shape

(80988, 13)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    x_array, y_array, test_size=0.4, shuffle=False, random_state=1004
)

In [6]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [7]:
image_w, image_h = X_train.shape[1], X_train.shape[2]

X_train = X_train.reshape(-1, image_w, image_h, 1)
X_test = X_test.reshape(-1, image_w, image_h, 1)

In [8]:
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(48592, 50, 50, 1) (32396, 50, 50, 1)
(48592, 13) (32396, 13)


In [9]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus: # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    print(f"{gpus[0]} 해당 GPU가 할당 되었습니다.")
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)

2024-02-27 16:02:00.567400: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-27 16:02:00.633604: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 16:02:00.633637: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 16:02:00.644780: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 16:02:00.673892: I tensorflow/core/platform/cpu_feature_guar

In [19]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12329579142380316319
xla_global_id: -1
]


In [11]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [15]:
import tensorflow as tf
import keras

model = keras.Sequential([
    keras.Input(shape=(50, 50, 1)),
    # keras.layers.Dense(1250, activation='relu'),
    # keras.layers.Dense(625, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(512, activation='relu'),
    # keras.layers.Dense(156, activation='relu'),
    # keras.layers.Dense(78, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu'),
    # keras.layers.Dense(19, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(32, activation='relu'),
    # keras.layers.Dense(19, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(13, activation='softmax'),
])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_4 (Dropout)         (None, 50, 50, 1)         0         
                                                                 
 dense_4 (Dense)             (None, 50, 50, 512)       1024      
                                                                 
 dropout_5 (Dropout)         (None, 50, 50, 512)       0         
                                                                 
 dense_5 (Dense)             (None, 50, 50, 64)        32832     
                                                                 
 dropout_6 (Dropout)         (None, 50, 50, 64)        0         
                                                                 
 dense_6 (Dense)             (None, 50, 50, 32)        2080      
                                                                 
 dropout_7 (Dropout)         (None, 50, 50, 32)       

In [18]:
batch_size = 128
epochs = 20

model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(0.05), metrics=["accuracy"])

history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test))

Epoch 1/20
380/380 [==============================] - 357s 935ms/step - loss: 2.0974 - accuracy: 0.1390 - val_loss: 10.9003 - val_accuracy: 0.0000e+00
Epoch 2/20
380/380 [==============================] - 354s 931ms/step - loss: 2.0976 - accuracy: 0.1388 - val_loss: 11.8268 - val_accuracy: 0.0000e+00
Epoch 3/20
380/380 [==============================] - 353s 930ms/step - loss: 2.0980 - accuracy: 0.1382 - val_loss: 12.3608 - val_accuracy: 0.0000e+00
Epoch 4/20
380/380 [==============================] - 355s 935ms/step - loss: 2.0977 - accuracy: 0.1403 - val_loss: 12.7060 - val_accuracy: 0.0000e+00
Epoch 5/20
256/380 [===================>..........] - ETA: 1:38 - loss: 2.0969 - accuracy: 0.1406

KeyboardInterrupt: 